<a id = "top"></a>

# Concert Tour Routing Plan Optimization
<font color = red>Implementing Lazy Constraints on MIP Models.</font>

***

#### $\bf{Problem\ description:}$ ${Case\ Study}$

&emsp;&emsp;A popular band is planning a tour across several cities, performing at different venues in each city. The band has a single plane to complete the tour across Canada and a single pilot to carry the band, staff, and the material used to set up the concert. The band has an unique objective for this tour: to deliver a different performance at each city. This requires the plane to carry all the material and instruments for each concert. Their current band manager is responsible for defining the order of the venues given the time windows at which the staff at the venue is available to set up the concert. During their last tour across the U.S., the band had multiple timing problems. Due to manual planning, all concerts in their last tour experienced delays, resulting in additional costs to hire extra staff at each venue. <u>The band wants to optimize their concert route for their tour across Canada to <b>minimize complete travel time and cost while ensuring that the staff time window is respected</b></u>.<br><br>

&emsp;&emsp;The band wants to determine the most efficient route structure for their pilot to reach each venue within the staff time window while ensuring that the tour starts and ends at the band hangar. The objective is to minimize the total distance traveled by the plane while ensuring that each venue location is visited once and that the plane capacity and staff time windows are respected.<br><br>

&emsp;&emsp;The band is asking our consulting firm to develop/create a proof of concept for an optimization software that can meet their requirements. The software will be used by the band manager to plan the
tour across Canada.<br><br>

#### $\bf{Data:}$ 
&emsp;&emsp;The band has a range of potential venues that they need to serve in the tour. The band also knows the time window at each venue, as well as the maximum capacity of the plane and the amount of material (demand) required at each venue.<br>

|    **Description**   | **Value** |
|:--------------------:|:---------:|
| Min number of venues |     80    |
| Max number of venues |    120    |
|   Demand per venue   |   1-4 Kg  |
|    Plane Capacity    |   500 Kg  |


#### $\bf{Note:}$ 
&emsp;&emsp;For this problem, the following assumptions are made:
- The travel time between locations is defined by the Euclidean distance between each location.
- The distance matrix is symmetrical, meaning that the distance between location $i$ and location $j$ is the same as the distance between location $j$ and location $i$.

<br>

### [$\bf{Click\ Here\ to\ Jump\ to\ the\ Implementation\ on\ Canadian\ Cities}$](#ca_implementation)
### [$\bf{Click\ Here\ to\ Jump\ to\ the\ Implementation\ on\ Synthetic\ Data}$](#synthetic_data)
### [$\bf{Click\ Here\ to\ Jump\ to\ the\ Implementation\ on\ Synthetic\ Data\ Considering\ Venue\ Availability}$](#synthetic_data_time)
<br><br>

<br>

***
<a id="ca_implementation"></a>
# $\bf{Implementation:}\ Canadian\ Cities$
In this attempt, we will optimize the routing plan <font color=red>to visit **ALL** top 150 Canadian cities</font> based on their population.
<div align = "right"><a href = '#top'>Back to Top</a></div>

## Import libraries and dataset
1. Import or install libraries and packages to use.
2. Read the Canadian Cities dataset.
3. A function **frame** is created to print the names.

In [1]:
import pandas as pd
import numpy as np

import gurobipy as gp
from gurobipy import *

import math
from itertools import combinations, permutations

import folium
import pprint
import matplotlib.pyplot as plt


# the frame() function is used to print a string with a frame.
def frame(title):
    print("\n"+"-"*(len(title)+4)+"\n| "+title+" |\n"+"-"*(len(title)+4))
    
# Define pretty print
pp = pprint.PrettyPrinter(indent = 1, compact = True)

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
filePath = "Datasets/"

# Read the file
ca_cities = pd.read_csv(filePath + "canadacities.csv")

# Transform city ranking to demand level
def ranking2demand(x):
    if x == 1:
        return 4
    elif x == 2:
        return 3
    elif x == 3:
        return 2
    elif x == 4:
        return 1
    else:
        "Error"
        
ca_cities["demand"] = ca_cities["ranking"].apply(lambda x: ranking2demand(x))

# Avoid cities with same names (Montreal -> Quebec_Montreal)
ca_cities["city"] = ca_cities[["province_name","city"]].agg("_".join, axis=1)

# Focus on top 150 cities by population
ca_cities = ca_cities.sort_values(by = "population", ascending = False)
ca_cities = ca_cities.head(150)

# Select columns of interest
ca_cities["city"] = ca_cities["city_ascii"]
ca_cities = ca_cities[["id","city","province_name","lat","lng","demand"]].set_index("city")

# Print the data
frame("canadacities.csv")
print("\nNumber of cities:", ca_cities.shape[0])
display(ca_cities.head())


--------------------
| canadacities.csv |
--------------------

Number of cities: 150


,id,province_name,lat,lng,demand
city,,,,,
Toronto,1124279679,Ontario,43.7417,-79.3733,4
Montreal,1124586170,Quebec,45.5089,-73.5617,4
Vancouver,1124825478,British Columbia,49.2500,-123.1000,4
Calgary,1124690423,Alberta,51.0500,-114.0667,4
Edmonton,1124290735,Alberta,53.5344,-113.4903,4


<br>

## Sets and Parameters

$i, j \in Cities $: indices and set of Canada top 150 cities cities based on population.

$\text{Pairings}= \{(i,j) \in Cities \times Cities \}$: Set of allowed pairings

$S \subset Cities$: A subset of the set of Canada top 150 citiesbased on population.

$G = (Cities, Pairings)$: A graph where the set $Cities$ defines the set of nodes and the set $Pairings$ defines the set of edges.

<br>

## Derive Distance Parameters
1 degree of longitude and latitude is approximately 111 kilometer.<br>
Here we use coordinates degree as our distance unit to avoid overcomplicating the formulation.

In [3]:
# Compute pairwise distance matrix
def distance(city1, city2):
    diff_lat = ca_cities.loc[city1]["lat"] - ca_cities.loc[city2]["lat"]
    diff_lng = ca_cities.loc[city1]["lng"] - ca_cities.loc[city2]["lng"]
    return math.sqrt(diff_lat**2 + diff_lng**2)

dist = {(c1, c2): distance(c1, c2) for c1, c2 in combinations(list(ca_cities.index), 2)}

<br>

## Modeling

As mentioned above in the original TSP example, our case is also a symmetric traveling salesman problem, we can make it more efficient by setting the object x[j,i] to x[i,j], instead of a constraint.

In [4]:
m = gp.Model("Concert Tour Routing Plan Optimization")

# Variables: is city 'i' adjacent to city 'j' on the tour?
vars = m.addVars(dist.keys(), obj=dist, vtype=GRB.BINARY, name='x')

# Symmetric direction: Copy the object
for i, j in vars.keys():
    vars[j, i] = vars[i, j]  # edge in opposite direction

# Constraints: two edges incident to each city
cons = m.addConstrs(vars.sum(c, '*') == 2 for c in ca_cities.index)

Using license file /Users/ylfaliang/gurobi.lic
Academic license - for non-commercial use only - expires 2024-04-08


<br>

## Callback Definition

Subtour constraints prevent multiple loops in a TSP tour. Because there are an exponential number of these constraints, we don't want to add them all to the model. Instead, we use a callback function to find violated subtour constraints and add them to the model as lazy constraints.

In [5]:
# Callback - use lazy constraints to eliminate sub-tours
def subtourelim(model, where):
    if where == GRB.Callback.MIPSOL:
        # make a list of edges selected in the solution
        vals = model.cbGetSolution(model._vars)
        selected = gp.tuplelist((i, j) for i, j in model._vars.keys()
                             if vals[i, j] > 0.5)
        # find the shortest cycle in the selected edge list
        tour = subtour(selected)
        if len(tour) < len(ca_cities.index):
            # add subtour elimination constr. for every pair of cities in subtour
            model.cbLazy(gp.quicksum(model._vars[i, j] for i, j in combinations(tour, 2))
                         <= len(tour)-1)

# Given a tuplelist of edges, find the shortest subtour
def subtour(edges):
    unvisited = list(ca_cities.index)[:]
    cycle = list(ca_cities.index)[:] # Dummy - guaranteed to be replaced
    while unvisited:  # true if list is non-empty
        thiscycle = []
        neighbors = unvisited
        while neighbors:
            current = neighbors[0]
            thiscycle.append(current)
            unvisited.remove(current)
            neighbors = [j for i, j in edges.select(current, '*')
                         if j in unvisited]
        if len(thiscycle) <= len(cycle):
            cycle = thiscycle # New shortest subtour
    return cycle

<br>

## Tune <a href="https://www.gurobi.com/documentation/10.0/refman/parameters.html" style="color:red">Gurobi Parameters</a> : MIP Cuts

These parameters affect the generation of MIP cutting planes. In all cases, a value of -1 corresponds to an automatic setting, which allows the solver to determine the appropriate level of aggressiveness in the cut generation. Unless otherwise noted, settings of 0, 1, and 2 correspond to no cut generation, conservative cut generation, or aggressive cut generation, respectively. The Cuts parameter provides global cut control, affecting the generation of all cuts. This parameter also has a setting of 3, which corresponds to very aggressive cut generation. The other parameters override the global Cuts parameter (so setting Cuts to 2 and CliqueCuts to 0 would generate all cut types aggressively, except clique cuts which would not be generated at all). 

In [6]:
#######################
## IMPORTANT NOTE
#######################

#REMEMBER TO RUN THE FOLLOWING COMMAND BEFORE TESTING OTHER PARAMETERS
#OTHERWISE GUROBI WILL USE ALL THE PREVIOUS PARAMETERS TO SOLVE THE
#MODEL

# Reset the values of the parameters
m.resetParams() # DO NOT COMMENT

#######################

# Run either Method A or Method B.

# A. Tune the model based on the selected parameters (see the parameter list).
m.setParam('FlowCoverCuts', 2)
#m.setParam('GomoryPasses', 2)
#m.setParam('RelaxLiftCuts', 2)

# WARNING: DO NOT USE AUTOMATIC TUNING WITH THE CHOSEN PARAMETERS
# B. Tune the complete formulation (automatic tunning)
#m.tune()

# Write the auto tuning results
#for i in range(m.tuneResultCount):
#    m.getTuneResult(i)
#    m.write('tune result_'+str(i)+'.prm')

Reset all parameters
Changed value of parameter FlowCoverCuts to 2
   Prev: -1  Min: -1  Max: 2  Default: -1


<br>

## Solve the Model

In [7]:
m._vars = vars
m.Params.lazyConstraints = 1
m.optimize(subtourelim)

Changed value of parameter lazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (mac64)
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 150 rows, 11175 columns and 22350 nonzeros
Model fingerprint: 0xc2185390
Variable types: 0 continuous, 11175 integer (11175 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-02, 7e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+00]
Presolve time: 0.02s
Presolved: 150 rows, 11175 columns, 22350 nonzeros
Variable types: 0 continuous, 11175 integer (11175 binary)

Root relaxation: objective 1.403792e+02, 230 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  140.37920    0   28          -  140.37920      -     -    0s
     0     0  140.93625    0   22      

<br>

## Analysis

We retrieve the optimal solution of the TSP and verify that the optimal route (or tour) goes to all the cities and returns to the origin city.

In [8]:
# Retrieve solution
vals = m.getAttr('x', vars)
selected = gp.tuplelist((i, j) for i, j in vals.keys() if vals[i, j] > 0.5)

tour = subtour(selected)
assert len(tour) == len(ca_cities)

In [9]:
province_color = {}
color_list = ["rgba(158, 1, 66, 1)",
              "rgba(213, 62, 79, 1)",
              "rgba(244, 109, 67, 1)",
              "rgba(253, 174, 97, 1)",
              "rgba(254, 224, 139, 1)",
              "rgba(230, 245, 152, 1)",
              "rgba(171, 221, 164, 1)",
              "rgba(102, 194, 165, 1)",
              "rgba(50, 136, 189, 1)",
              "rgba(94, 79, 162, 1)"]

for enum, p in enumerate(ca_cities.province_name.unique()):
    province_color[p] = color_list[enum]

In [10]:
# Map the solution
loc_center = [ca_cities.lat.mean(), ca_cities.lng.mean()]

ca_map = folium.Map(location = loc_center, tiles = "cartodbdark_matter" ,zoom_start = 4, control_scale = True)

points = []
for city in tour:
    city_lat = ca_cities.loc[city]["lat"]
    city_lng = ca_cities.loc[city]["lng"]
    province = ca_cities.loc[city]["province_name"]
    points.append((city_lat, city_lng))
    folium.CircleMarker([city_lat, city_lng], radius = int(ca_cities.loc[city]["demand"])**2.5, weight = 0, 
                        fill = True, fill_color = province_color[province], color = province_color[province],
                        tooltip="City: <b>"+city+"<b>").add_to(ca_map)
points.append(points[0])

folium.PolyLine(points, color = "silver").add_to(ca_map)
folium.TileLayer("cartodbpositron").add_to(ca_map)
folium.TileLayer("cartodbdark_matter").add_to(ca_map)
folium.LayerControl().add_to(ca_map)

ca_map